In [2]:
import os

In [3]:
os.chdir('../')
%pwd

'c:\\Users\\Suel.Abbasi\\Desktop\\DSA\\projects\\end to end ml project\\mental-health-tech-project\\mental-health-treatment-predictor'

In [5]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [27]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file
        )
        
        return data_ingestion_config

In [28]:
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size
import kagglehub
import shutil
import pandas as pd

In [29]:
# Download latest version
path = kagglehub.dataset_download("osmi/mental-health-in-tech-survey")

file_dir = os.path.join(path,'survey.csv')

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = kagglehub.dataset_download(dataset_url)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            source_dir = os.path.join(zip_download_dir,'survey.csv')
            destination_dir = os.path.join(self.config.local_data_file,'survey.csv')
            
            shutil.copy(source_dir, destination_dir)
            logger.info(f"Copying data from {source_dir} into file {destination_dir}")

        except Exception as e:
            raise e
        
    def read_file(self):
        path = os.path.join(self.config.local_data_file, 'survey.csv')
        try:
            df = pd.read_csv(path)
            return df
        except Exception as e:
            raise e

In [31]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    df = data_ingestion.read_file()
    print(len(df))
except Exception as e:
    raise e

[2025-02-11 13:39:33,912: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-11 13:39:33,921: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-11 13:39:33,922: INFO: common: created directory at: artifacts]
[2025-02-11 13:39:33,925: INFO: common: created directory at: artifacts/data_ingestion]
[2025-02-11 13:39:34,638: INFO: 739087031: Downloading data from osmi/mental-health-in-tech-survey into file C:\Users\Suel.Abbasi\.cache\kagglehub\datasets\osmi\mental-health-in-tech-survey\versions\3]
[2025-02-11 13:39:34,654: INFO: 739087031: Copying data from C:\Users\Suel.Abbasi\.cache\kagglehub\datasets\osmi\mental-health-in-tech-survey\versions\3\survey.csv into file artifacts/data_ingestion\survey.csv]
1259
